## Arguments
- db (default pmc)
- collection (default articles)
- skip_field (default none)
- extractors_to_use (default all)
- num_workers (default 4)

In [1]:
import os
os.chdir("..")

In [2]:
import multiprocessing as mp
import time
import sys
import pymongo
# from annotator.keyword_annotator import KeywordAnnotator
# from annotator.geoname_annotator import GeonameAnnotator
import pubcrawler.extractors as ex


def chunk_slices(length, by):
    items = list(range(0, length + 1, by))
    if length % by != 0:
        items.append(length)
    slices = [slice(items[i], items[i+1]) for i in range(0, len(items)-1)]
    return(slices)

def worker(url, db, collection, to_extract, query, slice_queue):
    articles = pymongo.MongoClient()[db][collection]
    for s in iter(slice_queue.get, 'STOP'):
        print(s)
        cursor = articles.find(query)[s]
        ex.extract_and_write_multiple(cursor, to_extract)

In [29]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    "-u", "--mongo_url", default="localhost", dest = "u"
)
parser.add_argument(
    "-d", "--mongo_db", default="pmc", dest = "d"
)
parser.add_argument(
    "-c", "--mongo_collection", default="articlesubset", dest = "c"
)
parser.add_argument(
    "-x", "-extract", action="append", default=None, dest = "x"
)
parser.add_argument(
    "-s", "-skip_field", default=None, dest = "s"
)
parser.add_argument(
    "-w", "-workers", default=4, dest = "w"
)
parser.add_argument(
    "-l", "-limit", default=None, dest = "l"
)
args = parser.parse_args(["-x", "extract_disease_ontology_keywords", "-s", "keywords", "-w", "8", "-c", "articlesubset"])
args

Namespace(c='articlesubset', d='pmc', l=None, s='keywords', u='localhost', w='8', x=['extract_disease_ontology_keywords'])

In [30]:
if args.x is not None:
    print(args.x)
    extractor_funs = [eval(x) for x in ['ex.' + x for x in args.x]]
else:
    print("Please specify at least one extractor function", file=sys.stderr)
    sys.exit(1)

['extract_disease_ontology_keywords']


In [31]:
if args.s is not None:
    query = {args.s: {'$exists': False}}
else:
    query = {}

In [32]:
print("Making connection.")
articles = pymongo.MongoClient(args.u)[args.d][args.c]

Making connection.


In [33]:
print("About to count.")
total_for_query = articles.count(query)
num_to_annotate = args.l if args.l is not None else total_for_query
num_workers = int(args.w)
print("Total for query is {}.".format(total_for_query))

print("About to chunk.")

About to count.
Total for query is 10000.
About to chunk.


In [27]:
ex.strip_article_info(articles)

In [1]:
queue = mp.Queue()
for i in chunk_slices(num_to_annotate, by = 100):
    queue.put(i)
for w in range(num_workers):
    queue.put('STOP')


worker_args = (
    args.u,
    args.d,
    args.c,
    extractor_funs,
    query,
    queue,
)

print("About to start.")

for w in range(num_workers):
    mp.Process(target=worker, args=worker_args).start()

In [3]:
import pubcrawler.extractors as extractors

In [5]:
extractors.extract_meta

<function pubcrawler.extractors.extract_meta>

In [7]:
eval('extract_meta')

NameError: name 'extract_meta' is not defined